## TODO
* Add predictions 
* adjust the event horizon beyond 1 (want to use 6 and 12)
* Auto identify max lags

In the dataset, every state has 11 coverage types. There are 5 main metrics for each coverage type. There are 69 months of data associated with each metric.

In [ ]:
whos

In [ ]:
%matplotlib inline

import matplotlib
import pandas as pd
from pprint import pprint
import numpy as np
import json

In [ ]:
%cd /home/kesj/work/claiment/
%ls

In [ ]:
infile= 'base2_CENT_02_2015.csv'

# import some of the helper functions

In [ ]:
def series_norm(series):
    return(series - series.mean())/series.std(ddof=1)


In [ ]:
def preprocess_cent_file(inputfile, x='STATE', y='COVERAGE', t='YEAR',
                         important_variables=[], omit_column=[], hdfs_flag=False, create_overall_variable=False):
    """
    A function that takes an input csv file of many timeseries data and processes it into the
    format that we want for CENT analysis.

    hdfs_flag indicates if the based file is in hdfs or not.

    the input parameters:
    data -- the flat 2d datashape
    important_variables -- the column name to use for the 'items' -- default is 'STATE'
    y -- the column name to use for the 'minor-axis' -- default is 'COVERAGE'
    t -- the column name to use for the 'major-axis' <- where timeseres go; default is 'YEAR'
    z -- the column title to use for 'labels' (these are the columns you can loop over)

    :returns
      my_grouped_df ( a grouped hierarchical data frame ready to be printed as key,value pairs
      important_variables ( the list of metrics used in the calculation)
      xvals (the list of the first major axis -- i.e. the states)
      yvals ( the list of the minor axis -- i.e. the coverages)
    """

    # if hdfs_flag:

    data = pd.read_csv(inputfile)
    # check format of a couple input formats. we want important_variables, omit_column to be lists

    if len(omit_column) != 0:  # omit a list of columns if so given
        data = data.drop(omit_column, axis=1, inplace=True)

    print "input dataframe has the dimensions of {0}".format(np.shape(data))
    print "the column titles are: {0}".format(data.columns)

    # check that x,y,t,variables_of_interest are in columns
    # create a set for the singletons:
    s1 = set([x, y, t])

    # create a set for all columns
    scolumns = set(data.columns.values)
    # test that s1 is subset of scolumns
    sdiff1 = s1 - scolumns
    if len(sdiff1) != 0:
        # have to fix the s1 input files
        print "We have a problem because the columns you want to pivot on are not present"
        print sdiff1, " is missing from ", scolumns
        return 0
    else:
        # convert the year to a date-time object
        data[t] = pd.to_datetime(data[t])

    if len(important_variables) == 0:  # define these based upon the input data if not defined
        important_variables = list(scolumns - s1)
        print "important variables are determined by the input data."
        # print "{0} are the important variables determined based upon the input data".format(important_variables)


        # option to generate an overall variable that is the normalized average of the other variables of interest
    if create_overall_variable:
        data['Overall'] = data.groupby([x, y]).transform(series_norm)[important_variables].sum(axis=1)
        important_variables += ['Overall']

    # generate list of unique values for the 3 input parameters: x,y,t
    print s1

    xvals = data[x].unique()
    yvals = data[y].unique()
    tvals = data[t].unique()

    print len(xvals), len(yvals), len(tvals), len(important_variables)
    # to do: check that all columns are used
    # to do: reshape the 2d data frame into a multi-index, hierarchical df
    my_grouped_data = data.groupby((x, y))  # set_index([x,y,t]
    return (my_grouped_data, important_variables, xvals, yvals)


In [ ]:
#helper functions
def interpolate_CENT_predictions(input_list,round_digit=2):
    """Returns list of interpolated values, round to the number of places given
       strictly assumes spacing of 1 month, 6 month and 12 month.
    """
    interpolated_predictions=[]
    forecast_1_month = input_list[0]
    forecast_6_month = input_list[1]
    forecast_12_month = input_list[2]
    # Add 1 month prediction
    interpolated_predictions.append(round(forecast_1_month,round_digit))

    # Interpolate values between 1 month and 6 months
    one_six_gap = (forecast_6_month-forecast_1_month)/5.0
    for e in range(1,5):
        interpolated_predictions.append(round(forecast_1_month+e*one_six_gap,round_digit))

    # Add 6 month prediction
    interpolated_predictions.append(round(forecast_6_month,round_digit))

    # Interpolate values between 6 months and 12 months
    six_twelve_gap = (forecast_12_month-forecast_6_month)/6.0
    for e in range(1,6):
        interpolated_predictions.append(round(forecast_6_month+e*six_twelve_gap,round_digit))

    # Add 12 month prediction
    interpolated_predictions.append(round(forecast_12_month,round_digit))
    return interpolated_predictions

def make_interpolated_forecasts(x):
    """
    function to generate the interpolated values from a list of values and errors
    :param x: list of 3 values and 3 errors in the format of [v1,e1,v2,e2,v3,e3]
    :return: series of forecasted_values and forecasted_errors for the 12 months
    assuming v1 is 1 month value, v2 is 6 month value and v3 is 12 month valaue
    """
    # if the input values are nan, just return an empty list
    if sum(np.isnan(x)) == len(x) :
        forecast_values = []
        forecast_errors = []
    else:
        values = x[::2]
        errors = x[1::2]
        forecast_values = interpolate_CENT_predictions(values)
        forecast_errors = interpolate_CENT_predictions(errors)

    return pd.Series(dict(forecast=forecast_values, std=forecast_errors))


In [ ]:
### function to initialize API data structure containers
def initialize_CENT_api_containers(coverages, variables_of_interest, states, time_horizons, start_year=2007, start_month=0):
    """
    Function for initializing the CENT api containers
    :param coverages: list of possible coverages
    :param variables_of_interest: list of variables of interest
    :param states: list of states
    :param time_horizons --> list of time horizons
    :param start_year: starting year
    :param start_month: starting month
    :return: detail_data, overview_data, and panel
    """

    # create a 4D panel data frame for the forecast data
    forecast_dimension = []
    for time in time_horizons:
        forecast_dimension.append('v'+str(time))
        forecast_dimension.append('e'+str(time))

    my_panel = pd.Panel4D(labels=states, items=coverages,major_axis=variables_of_interest,minor_axis=forecast_dimension)

    # overview_data is of the format overview_data[horizon][metric][variable][coverage][state]
    overview_data = {}
    for horizon in time_horizons:
        overview_data[horizon] = {"startYear": start_year,
                                  "startMonth": start_month,
                                  "metrics": {}}
        for variable in variables_of_interest:
            overview_data[horizon]["metrics"][variable] = {}
            for coverage in coverages:
                overview_data[horizon]["metrics"][variable][coverage] = {}
                for state in states:
                    overview_data[horizon]["metrics"][variable][coverage][state] = []

    # detail_data is of the format of detail_data[horizon][state]["metrics"][variable][coverage]
    detail_data = {}
    for horizon in time_horizons:
        detail_data[horizon] = {}
        for state in states:
            detail_data[horizon][state] = {"startYear": start_year,
                              "startMonth": start_month,
                              "metrics": {}}
            for variable in variables_of_interest:
                detail_data[horizon][state]["metrics"][variable] = {}
                for coverage in coverages:
                    detail_data[horizon][state]["metrics"][variable][coverage] = {}


    return (detail_data, overview_data, my_panel)


In [ ]:
def build_models(variables_of_interest,grouped_df,my_time_horizons,my_panel,detail_data,overview_data):
    # arrays for the errors
    error_dict = {}
    for time_horizon in my_time_horizons:
        error_dict[time_horizon] = []
    
    from collections import defaultdict
    rnd_digits = defaultdict(int)
    for v in variables_of_interest:
        if v.endswith('Count'):
            rnd_digits[v]=0
        elif v.endswith('CNT'):
            rnd_digits[v]=0
        else:
            rnd_digits[v]=2
    # display progress
    total_groups = len(grouped_df.groups)
    ngrps=0
    for g in grouped_df.groups:
        state = g[0]
        coverage = g[1]
        focused_data = grouped_df.get_group(g)
        ngrps+=1
        print state, coverage,"......",ngrps," out of ",total_groups
        for time_horizon in my_time_horizons:
            X, y_data, x, undiff = time_series_to_cross_section(focused_data[variables_of_interest],
                                                        forecast_horizon=time_horizon,
                                                        max_fair_lags=13,
                                                        seasonal_factor=12)
            for variable in variables_of_interest:
                # Account for time series with no variation
                state_actuals = focused_data[variable]
                std_of_actuals = state_actuals.std(ddof=1)
                # code to deal with forecast data
                value_col = 'v'+str(time_horizon)
                error_col = 'e'+str(time_horizon)
                if std_of_actuals == 0:
                    overview_data[time_horizon]["metrics"][variable][coverage][state] = [0]*len(state_actuals)
                    detail_data[time_horizon][state]["metrics"][variable][coverage] = {"stddev": 0,
                                                                     "actual": list(state_actuals),
                                                                     "predicted" : list(state_actuals)}
                    my_panel[state][coverage].loc[variable,value_col] = 0#point_forecast[0]
                    my_panel[state][coverage].loc[variable,error_col] = 0#point_forecast[1]                                                     "predicted": list(state_actuals)}
                    continue
            
                # Build custom model for dataset. This is the line of code that takes all the time.
                final_model, variables_in_model = optimized_rf(X, y_data[variable],
                                                       variable_importance_n_estimators=50,
                                                       n_estimators_in_grid_search=20,
                                                       number_of_important_variables_to_use_options=[6,8],#[6],#, 8, 10, 12, 15, 20],
                                                       variable_importance_max_features_options=['sqrt'],#, 0.5, .75, 'auto'],
                                                       n_estimators_to_retrain_best_model=50,
                                                       verbose=False, n_random_models_to_test=6,#3,
                                                       charts=False, n_jobs=1)



                state_predictions = undiff(final_model.oob_prediction_, variable, True)
                state_residuals = focused_data[variable] - state_predictions
                data_consumed_for_model = sum(state_residuals == 0)
                std_of_residuals = state_residuals[data_consumed_for_model:].std(ddof=1)
                state_std_residuals = state_residuals/std_of_residuals
                abs_average_error = abs((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals).mean()
                MSE = (((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals)**2).mean()         

                #  point forecast for this variable and time horizon
                forecast = model_forecast(x,variable, final_model, variables_in_model,undiff)
                #convert list of values to 2 floating point digits or 0 for counts
                rnd_size = rnd_digits[variable]
                if rnd_size > 0 :
                    state_actuals = [ round(elem,rnd_size) for elem in list(state_actuals)]
                    state_predictions = [ round(elem,rnd_size) for elem in list(state_predictions)]
                else:
                    state_actuals = [ int(round(elem,0)) for elem in list(state_actuals)]
                    state_predictions = [ int(round(elem,0)) for elem in list(state_predictions)]
            
                overview_values = [round(elem,2) for elem in list(state_std_residuals.values)]
        
                # Update primary data structures with model results
                overview_data[time_horizon]["metrics"][variable][coverage][state] = overview_values
                detail_data[time_horizon][state]["metrics"][variable][coverage] = {"stddev": round(std_of_residuals,5),
                                                                 "actual": state_actuals,
                                                                 "predicted": state_predictions}
                MSE = round(MSE,5)
                abs_average_error = round(abs_average_error,5)
                my_panel[state][coverage].loc[variable,value_col] = round(forecast,5)
                my_panel[state][coverage].loc[variable,error_col] = round(std_of_residuals,5)
                error_dict[time_horizon].append([MSE,abs_average_error])
                
        #print '********************************************************************************************'
    #print total_absolute_average_error / float(models_built), total_mean_squared_error / float(models_built)
    #print models_built
    #print '********************************************************************************************'
    return(my_panel,detail_data,overview_data,error_dict)

In [ ]:
def calculate_overall_error(error_dict):
    for time in error_dict.keys():
        nmodels = len(error_dict[time])
        total_mse = 0.0
        total_abserr = 0.0
        n=0
        for model_error in error_dict[time]:
            mse = model_error[0]
            abserr = model_error[1]
            if not np.isnan(mse) and  not np.isnan(abserr):
                total_mse += mse
                n+=1
                total_abserr += abserr
            
            #if time == 1:
            #    print mse, abserr, n
        
        print '********************************************************************************************'
        print time, total_abserr / float(n), total_mse / float(n)
        print n, nmodels
        print '********************************************************************************************'    
    return
        

In [ ]:
def CENT_rolling_12(df, columns_for_r12=["Reported Count", "Paid Count", "Pending Count", "Indemnity", "Severity","CIF","ALAE","CWP","OIE_CNT","SUIT_CNT"]):
    vvv = df.copy()
    result_list_of_df = []

    for state, state_data in vvv.groupby("STATE"):
        for coverage, coverage_data in state_data.groupby("COVERAGE"):
            for metrics in columns_for_r12:
                coverage_data[metrics] = pd.rolling_mean(coverage_data[metrics], window=12)
            result_list_of_df.append(coverage_data)

    vvv = pd.concat(result_list_of_df)
    vvv.dropna(inplace=True)
    return vvv

#CENT_rolling_12(vv)

In [ ]:
## TOP K from a dictionary
def add_value_to_dictionary(overview_dict,time_horizon,variable,coverage,joined_dictionary, index_to_add = -1):
    """
    Function to convert the overview dictionary data into a flattened_dictionary with the keys joined as a tuple.
    :param overview_dict: input dictionary of overview_data
    :param time_horizon: specific time_horizon
    :param variable: specific variable (of interest)
    :param coverage: specific coverage
    :param joined_dictionary: dictionary of flattened values of last data.
    :param index_to_add: which value to add to this flattened dictionary; default is most recent value.
    :return:
    """
    # if no input dictionary given, create one.
    if joined_dictionary == None:
        joined_dictionary = {}

    for key,value in overview_dict[time_horizon]['metrics'][variable][coverage].iteritems():
        long_key = (time_horizon,variable,coverage,key)
        try :
            value_to_add = value[index_to_add]
            joined_dictionary[long_key]=value_to_add
        except IndexError:
            #print key, " lacks values."
            # skip over cases where there are no values.
            pass
    return joined_dictionary

def return_top_bottom_K(flattened_dictionary, my_column_list = ['horizon','Variable','Coverage','State','Value'], K=10):
    """
    Function to return the top and bottom K from a flattened dictionary
    :param flattened_dictionary: flattened dictionary where the key is the 1st N-1 elements of my_column_list and the
    value is the last one.
    :param K: number of elements from top and bottom to return
    :return: table_df, a pandas data frame of the topK and bottomK results.
    """
    sorted_dictionary = sorted(flattened_dictionary, key=flattened_dictionary.get)

    # create an empty dataframe
    table_df = pd.DataFrame(data=np.zeros((0,len(my_column_list))), columns=my_column_list)

    table_size = len(table_df)
    for key in sorted_dictionary[:K]:
        key_list = [item for item in key]
        key_list.append(flattened_dictionary[key])
        table_df.loc[table_size]=key_list
        table_size+=1
        #print key_list

    for key in sorted_dictionary[-1*K:]:
        key_list = [item for item in key]
        key_list.append(flattened_dictionary[key])
        table_df.loc[table_size]=key_list
        table_size+=1
        #print key_list

    return table_df

In [ ]:
def clean_up_results_serial(my_panel,detail_data,overview_data,time_horizon_list=[1,6,12]):
    # switch the ordering of the axes to match detail_data ordering
    altered_panel = my_panel.swapaxes('items','major_axis')
    for label in altered_panel.labels:
        for item in altered_panel[label].items:
            forecasted = altered_panel[label][item].apply(lambda x: make_interpolated_forecasts(x),axis=1)
    #        altered_panel[label][item].loc[forecasted.index,'forecast']=r['forecast'].values
    #        altered_panel[label][item].loc[forecasted.index,'std']=r['std'].values
            ## now output the forecast to the horizon = 1 key = 'forecast'
            for index_cov in forecasted.index:
            #append a new dictionary level for 1st time_horizon
                detail_data[1][label]['metrics'][item][index_cov]['forecast'] = {
                    'values':forecasted.ix[index_cov]['forecast'],
                    'std':forecasted.ix[index_cov]['std'] }

    # augment the overview_data with the table info for topK and bottomK
    # create a table of the overview results
    states = list(my_panel.labels)
    coverages = list(my_panel.items)
    importance_variables = list(my_panel.major_axis)

    for time in time_horizon_list:
        flattened_overview_dict = {}
        for variable in importance_variables:
            for coverage in coverages:
                flattened_overview_dict = add_value_to_dictionary(overview_data, time, variable, coverage,
                                                                  flattened_overview_dict)

        table_df = return_top_bottom_K(flattened_overview_dict)
        # sort by value;
        table_df.sort(['Value'],inplace=True,ascending=False)
        # append this information to overview_data
        overview_data[time]['table'] = table_df[['State','Coverage','Variable','Value']].values.tolist()

    # check that tgtdir ends in '/'; append if not
    #if not tgtdir.endswith('/'):
    #    tgtdir += '/'

    #overview_file = tgtdir + 'overview.json'
    #detail_file = tgtdir + 'detail.json'
    overview_file = 'overview.json'#_',+fname+'.json'
    detail_file = 'detail.json'#_'+fname+'.json'
    
    # check on existence of the directory and create it if missing
    #make_sure_path_exists(tgtdir)
    with open(overview_file, 'w') as outfile:
        json.dump(overview_data, outfile)

    with open(detail_file, 'w') as outfile:
        json.dump(detail_data, outfile)

    return

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

def number_of_nulls(series):
    return series.isnull().sum()

def are_the_last_n_values_null(series, n):
    """
    series: A selection of a pandas dataframe
    n: If the last n values are null, returns True
    """
    number_of_null_values = number_of_nulls(series)
    return number_of_null_values == series[-number_of_null_values:].isnull().sum()

def n_of_missing_last_values(series):
    number_of_null_values = number_of_nulls(series)
    if number_of_null_values == 0:
        return 0
    while True:
        if are_the_last_n_values_null(series, number_of_null_values):
            return number_of_null_values
        number_of_null_values -= 1
        if number_of_null_values < 0:
            return "Error"
        
def missing_last_values(df):
    """Returns a Series of each variable with
    the number of missing values at the end
    """
    missing_last_values_df = pd.Series()
    for variable in df.columns:
        missing_last_values_df[variable] = n_of_missing_last_values(df[variable])
    return missing_last_values_df

def make_lag(series, n_lag):
    """OLD VERSION. Returns a series with specified lag"""
    original_index = series.index
    series = series[:-n_lag]
    series.index = original_index[n_lag:]
    series = series.reindex(original_index)
    return series

# def make_lag(series, n_lag):
#     """Returns a series with specified lag"""
#     # Update index
#     max_index_value = series.index[-1]
#     index_to_add = ["t%d"%(e+1) for e in xrange(n_lag)]
#     new_index = np.concatenate([series.index, index_to_add])
    
#     # Update values
#     values_to_add = [np.nan]*n_lag
#     new_values = np.concatenate([values_to_add, series.values])
    
#     return pd.Series(new_values, new_index)

def difference_data(df, diff=1):
    """
    df: pandas df
        Must be sorted with the most recent data at the bottom of the 
        dataframe
        
    diff: integer
        Default of 1 implies first difference. If 2, it will be a second 
        difference
    """        
    data_old = df.copy()
    df = df[:-diff]
    df.index = data_old.index[diff:]
    df = df.reindex(data_old.index)
    return data_old - df

def time_series_to_cross_section(df, forecast_horizon=1, 
                                 difference=True, 
                                 max_fair_lags=4, seasonal_factor=None):
    """
    df: pandas df or path to csv file
        There should be no missing values within the series. Missing
        values at the end or beginning of the series are okay.
    
    forecast_horizon: integer
        Specifies how far ahead to make the forecast from the most
        recent NON-NULL value.
        
    difference: boolean
        If True, all the data is differenced at the forecast_horizon.
        For example, if the forecast horizon is 6, then t0 will be
        subtracted from t6. The reason for this is if we forecast the 
        simple difference (t1 - t0), we will not be able to
        undifference it.
        
    max_fair_lags: integer
        Will use up to max_fair_lags lags of data. If you enter 6 lags
        and the last 4 data points of a series are missing, only
        lag 6 will be included (lag 5 is only fair to predict the 
        most recent y value)
        
    seasonal_factor: integer
        Adds a column to the dataset that is 0, 1, ... , 
        seasonal_factor-1 repeating.
        
    Returns:
    output_df: pandas df
        A fair design matrix to use to predict any of the columns of
        the output_y_df
        
    output_y_df: pandas df
        Contains a df with fair y values to use for every column in 
        the df. Simply select the column data and use it as y in any
        ML algorithm paired with the output_df as X.
        
    x_to_predict: pandas series
        Once you build your model and want to predict the next point,
        use this data in the predict method of the model. This point 
        will correspond to the forecast horizon relative to the most 
        recent valid data point.
        
    undiff_x_predict_function: function
        The predict series of the model will return the expected change
        and not the absolute value of the prediction. Use this function
        to undifference the prediction. The input is the differenced
        prediction and the column name.
    """
    # Create a df that specifies the most recent fair lag
    # that can be used for each column
    missing_last_values_df = missing_last_values(df)
    most_recent_fair_lag = missing_last_values_df + forecast_horizon
    
    # Specify output dataframe
    output_df = pd.DataFrame()
    
    # Add all fair lags to dataframe
    for var in df.columns:
        min_lag = most_recent_fair_lag[var]
        # print var, min_lag, max_fair_lags
        while min_lag <= max_fair_lags:
            output_df["%s_lag%d" % (var, min_lag)] = make_lag(df[var], 
                                                              min_lag)
            min_lag += 1
    
    # Differenced design matrix
    if difference:
        output_df = difference_data(output_df, forecast_horizon)
    
    # Drop rows with NaN
    output_df.dropna(inplace=True)

    # Align df of y values so that it is fair for each variable
    output_y_df = df.copy()
    for var in df.columns:
        recent_missing = missing_last_values_df[var]
        if recent_missing > 0:
            output_y_df[var] = np.concatenate([list([np.nan])*recent_missing, 
                                              output_y_df[var][:-recent_missing]])
    
    if difference:
        undiff_output_y_df = output_y_df.copy()
        output_y_df = difference_data(output_y_df, forecast_horizon)
        output_y_df = output_y_df[forecast_horizon:]
        
    # Calculate number of rows to drop from output_y_df
    rows_to_drop = output_y_df.shape[0] - output_df.shape[0] + forecast_horizon # Adjusts for t1-tn
    output_y_df = output_y_df[rows_to_drop:]
    
    # Add seasonal factor
    if seasonal_factor:
        output_df["seasonality"] = np.array(range(seasonal_factor)*(output_df.shape[0]/seasonal_factor+1))[:output_df.shape[0]]
    
    # Copy the last line as the x to predict
    x_to_predict = pd.Series(output_df[-1:].values[0], index=output_df.columns)
    
    # Drop the last rows of output_df
    output_df = output_df[:-forecast_horizon]
    
    # Make function that undifferences the data
    def undifference(differenced_data, column_name, include_initial_points=False):
        """
        Undifferences either the in-sample predictions or the 
        out-of-sample prediction.
        
        differenced_data: pandas dataframe or pandas series
        If dataframe, the column_name must be the same between the original 
        dataset and the original data. If series, it doesn't matter
        
        column_name: string
        The name of the column to undifference
        
        include_initial_points: Boolean, optional (default=False)
        If True, would prepend the series with the actual values. These
        values should not be considered to be fair predictions.
        
        Returns
        Undifferenced series or point.
        """
        original_series = df[column_name].values
        
        # If only a single data point is provided, return single 
        # undifferenced value
        try:
            differenced_data = float(differenced_data)
        except TypeError:
            pass
        
        if type(differenced_data) in [int, float] or len(differenced_data) == 1:
            if type(differenced_data) == list:
                differenced_data = differenced_data[0]
            return original_series[-1] + differenced_data
        
        if len(differenced_data.shape) == 2:
            differenced_data = differenced_data[column_name]
        len_of_differenced_data = differenced_data.shape[0]
        len_of_original_series = original_series.shape[0]
        size_diff = len_of_original_series - len_of_differenced_data
        
        # Drop rows that were removed due to lags
        new_series = original_series[(size_diff - forecast_horizon):-forecast_horizon]
        
        undifferenced_series = new_series + differenced_data
        
        if include_initial_points:
            undifferenced_series = np.concatenate((original_series[:size_diff], undifferenced_series), 
                                                  axis=0)
        return undifferenced_series
    
    return output_df, output_y_df, x_to_predict, undifference
    
    
def sort_variables_by_importance(X, y, n_estimators=100, 
                                 max_features="auto", chart=True,
                                 random_state=42, n_jobs=1):
    """Returns an array of column names sorted by rf importance"""
    model = RandomForestRegressor(n_estimators, 
                                  max_features=max_features, 
                                  random_state=random_state, 
                                  n_jobs=n_jobs)
    model.fit(X, y)
    important_variables = pd.Series(model.feature_importances_, 
                                    index=X.columns)
    important_variables.sort()
    if chart:
        important_variables.plot(kind="barh", figsize=(5,15))
        plt.show()
    important_variables.sort(ascending=False)
    return list(important_variables.index)


def oob_randomized_rf_gridsearch(X, y, n_estimators=50, 
                                 n_models_to_test=10, 
                                 top_models_to_print=3, 
                                 trees_in_final_model="auto",
                                 verbose=True, 
                                 random_state=42, 
                                 n_jobs=1):
    """
    Performs a simple randomized gridsearch using the oob_errors.
    
    Parameters
    ----------
    X: pandas df
    
    y: pandas Series
    
    n_estimators: int, optional (default=50)
    Number of trees in random forest
    
    n_models_to_test: int, optional (default=10)
    Number of unique parameter combinations to test
    
    top_models_to_print: int, optional (default=3)
    Prints the parameter list of the top models selected
    
    trees_in_final_model: int,  "auto", or None, optional (default="auto")
    After finding the optimal model, retrains model with this many
    trees. If "auto", is equal to n_estimators * n_models_to_test/2.
    If None, doesn't train final model. Returns trained model list 
    instead.
    
    verbose: Boolean, optional (default=True)
    Prints final trained model stats if True
    
    Returns 
    -------
    Final trained model
    """
    max_features_options = [.1, "sqrt", .25, .5, .75, 1.0]
    min_samples_split_options = [2, 4, 6]
    min_samples_leaf_options = [1, 2, 3]
    
    max_trials = len(max_features_options)*len(min_samples_split_options)*len(min_samples_leaf_options)
    if n_models_to_test >= max_trials:
        print "Too many trials, n_models_to_test set to", max_trials - 1
        n_models_to_test = max_trials - 1
    
    def get_parameters():
        max_features = np.random.choice(max_features_options, 1, p=[.2, .1, .1, .1, .1, .4])[0]
        if max_features != "sqrt":
            max_features = float(max_features)
        min_samples_split = np.random.choice(min_samples_split_options, 1, p=[.7, .2, .1])[0]
        min_samples_leaf = np.random.choice(min_samples_leaf_options, 1, p=[.7, .2, .1])[0]
        return max_features, min_samples_split, min_samples_leaf
    
    parameters_tested = []
    model_results = []
    for trial in xrange(n_models_to_test):
        # Ensure only unique parameter combinations are tested
        while True:
            max_features, min_samples_split, min_samples_leaf = get_parameters()
            if (max_features, min_samples_split, min_samples_leaf) not in parameters_tested:
                parameters_tested += [(max_features, min_samples_split, min_samples_leaf)]
                break

        model = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features, 
                                      min_samples_split=min_samples_split, 
                                      min_samples_leaf=min_samples_leaf, oob_score=True,
                                      random_state=random_state, n_jobs=n_jobs)
        try:
            model.fit(X, y)
            model_specs = zip(["max_features", "min_samples_split", "min_samples_leaf"], 
                              (max_features, min_samples_split, min_samples_leaf))
            model_results += [(round(model.oob_score_,4), model_specs)]
        except ValueError:
            pass
        
    model_results.sort(reverse=True)
    
    if top_models_to_print:
        pprint(model_results[:top_models_to_print])
        
    if trees_in_final_model:
        if trees_in_final_model == "auto":
            trees_in_final_model = int(n_estimators*n_models_to_test/2.0)
            
        top_model = model_results[0][1]
        max_features, min_samples_split, min_samples_leaf = [e[1] for e in top_model]
        model = RandomForestRegressor(n_estimators=trees_in_final_model, 
                                      max_features=max_features, 
                                      min_samples_split=min_samples_split, 
                                      min_samples_leaf=min_samples_leaf, 
                                      oob_score=True,
                                      random_state=random_state, 
                                      n_jobs=n_jobs)
        model.fit(X, y)
        if verbose:
            print ""
            print "Final model results"
            print (round(model.oob_score_,4), top_model)
        return model
    else:
        return model_results
    
    return None
  
def optimized_rf(X, y, variable_importance_n_estimators=100, 
                 variable_importance_max_features_options=["sqrt", .5, "auto"],
                 number_of_important_variables_to_use_options=[8, 10, 12, 15],
                 n_estimators_in_grid_search=50,
                 n_estimators_to_retrain_best_model=100,
                 n_random_models_to_test=15,
                 verbose=True,
                 charts=False,
                 random_state=42,
                 n_jobs=1):
    """
    Auto optimize the RF oob R^2 using a randomized search
    
    Returns
    -------
    Optimized model, variables_in_model
    """
    np.random.seed(seed=random_state)
    best_model_score = -100
    
    for max_features in variable_importance_max_features_options:
        if verbose:
            print
            print "*"*60
            print "Variable importance optimization max features:", max_features
            print "*"*60
        important_variables = sort_variables_by_importance(X, y, 
                                                           n_estimators=variable_importance_n_estimators, 
                                                           max_features=max_features, 
                                                           chart=charts)

        for n_important_variables_to_use in number_of_important_variables_to_use_options:
            if n_important_variables_to_use > len(X.columns):
                break
            var_to_use = important_variables[:n_important_variables_to_use]
            model = oob_randomized_rf_gridsearch(X[var_to_use], y, 
                                                 n_estimators=n_estimators_in_grid_search, 
                                                 n_models_to_test=n_random_models_to_test, 
                                                 top_models_to_print=None, 
                                                 trees_in_final_model=n_estimators_to_retrain_best_model,
                                                 verbose=verbose, 
                                                 n_jobs=n_jobs)
            if model.oob_score_ > best_model_score:
                best_model = model
                best_var_importance_max_features = max_features
                variables_in_model = var_to_use
                best_model_score = model.oob_score_
    return best_model, variables_in_model

def model_forecast(x_to_predict,variable, mdl,mdl_vars, undiff):
    """

    :param x_to_predict: This is the x value for the given point in the future to predict
    :param variable: the current variable (metric) of interest to forcast ahead
    :param mdl: the optimized model returned for a given series and horizon
    :param mdl_vars: the corresponding variables used in that model
    :param undiff: the function used to transform from differenced to absolute values
    :return: the point prediciton (forecast) corresponding to the associated future time point.
    """
    raw_predict = mdl.predict(x_to_predict[mdl_vars])
    forecast = undiff(raw_predict, variable, True)
    return forecast

# Preprocess the data from CENT

In [ ]:
data = pd.read_csv(infile)
np.shape(data)

In [ ]:
processed_df, variables, states, coverages = preprocess_cent_file(infile)

In [ ]:
#checking this file is present 
processed_df.get_group(('VERMONT','MPC'))

In [ ]:
#checking this file is present 
processed_df.get_group(('VERMONT','PIP'))

In [ ]:
my_time_horizons = [1,6,12]

In [ ]:
# initialize api containers
detail_data, overview_data, my_panel = initialize_CENT_api_containers(coverages, variables,
                                                                          states, time_horizons=my_time_horizons)

# build models

In [ ]:
myp3,detail3,overview3,errortallies = build_models(variables,processed_df,my_time_horizons,my_panel,detail_data,overview_data)

#now clean up the results and make forecasts

### first check on the vermont data:


In [ ]:
for variable in variables:
    print detail3[1]['VERMONT']['metrics'][variable]['MPC'].keys()

In [ ]:
plt.scatter(detail3[1]['VERMONT']['metrics'][variable]['MPC']['actual'],detail3[1]['VERMONT']['metrics'][variable]['MPC']['predicted'])
plt.xlabel('actual')
plt.ylabel('predicted')

In [ ]:
plt.scatter(detail3[1]['VERMONT']['metrics'][variables[-1]]['MPC']['actual'],detail3[1]['VERMONT']['metrics'][variables[-1]]['MPC']['predicted'],alpha=0.3,color='steelblue')
plt.xlabel('actual')
plt.ylabel('predicted')

In [ ]:
myp3

In [ ]:
calculate_overall_error(errortallies)

In [ ]:
%pwd

In [ ]:
!mkdir serial_output_3
%cd serial_output_3

In [ ]:
clean_up_results_serial(myp3,detail3,overview3,[1,6,12])

# DEAL with ROLLING 12

In [ ]:
%cd ../
!mkdir serial_output_3_R12
dd = pd.read_csv(infile)
r12df = CENT_rolling_12(dd,variables)

### Create a grouped Rolling12 data frame and pass through the same procedure

In [ ]:
grouped12 = r12df.groupby(('STATE','COVERAGE'))

In [ ]:
# create the data structures
Rdetail_data, Roverview_data, Rmy_panel = initialize_CENT_api_containers(coverages, variables,
                                                                          states, time_horizons=my_time_horizons)
# generate the models
Rmyp3,Rdetail3,Roverview3,Rerrortallies = build_models(variables,grouped12,my_time_horizons,Rmy_panel,Rdetail_data,Roverview_data)

In [ ]:
#calculate overall error
calculate_overall_error(Rerrortallies)

In [ ]:
# save output
%cd serial_output_3_R12/

In [ ]:
#clean-up the results
clean_up_results_serial(Rmyp3,Rdetail3,Roverview3,[1,6,12])

### STOP HERE

## function to analyze the results

In [ ]:
def extract_detail_data(detail,time_horizon,state,coverage,metric):
    data = detail[time_horizon][state]['metrics'][metric][coverage]
    start_month = detail[time_horizon][state]['startMonth']
    start_year = detail[time_horizon][state]['startYear']
    actual = data['actual']
    predicted = data['predicted']
    std = data['stddev']
    nmonths = len(actual)
    forecast_values = list(np.zeros((12,)))
    forecast_errors = list(np.zeros((12,)))
    if time_horizon == 1:
        forecast_values = data['forecast']['values']
        forecast_errors = data['forecast']['std']
    #if r12_flag
    return actual,predicted,std,forecast_values, forecast_errors

In [ ]:
def detail_to_data_frame(detail,time_horizon,state,coverage,metric,r12_flag=False):
    data = detail[time_horizon][state]['metrics'][metric][coverage]
    start_month = detail[time_horizon][state]['startMonth']
    start_year = detail[time_horizon][state]['startYear']
    actual = data['actual']
    predicted = data['predicted']
    std = data['stddev']
    nmonths = len(actual)
    forecast_values = list(np.zeros((12,)))
    forecast_errors = list(np.zeros((12,)))
    if time_horizon == 1:
        forecast_values = data['forecast']['values']
        forecast_errors = data['forecast']['std']
        #nmonths+=12
    
    if r12_flag:
        start_date = str(start_month+12)+'-'+str(start_year)
        date_index = pd.date_range(start_date,freq='M',periods=nmonths)
    else:
        start_date = str(start_month+1)+'-'+str(start_year)
        date_index = pd.date_range(start_date,freq='M',periods=nmonths)
    
    df = pd.DataFrame(index = date_index)
    df['actual'] = actual
    df['predicted'] = predicted
    
    return df,std,forecast_values,forecast_errors

In [ ]:
processed_df.get_group(('VERMONT','MPC'))

In [ ]:
detail3[1]['MISSOURI']['metrics']['Paid Count']['PIP']['forecast']#.keys()

In [ ]:
a,p,std, fv,fe = extract_detail_data(detail3,1,'MISSOURI','PIP','Paid Count')

In [ ]:
plt.plot(a,'k',lw=2)
plt.errorbar(np.arange(0,len(p)),p,1.96*std)

In [ ]:
plt.plot(a,'k')
plt.errorbar(np.arange(0,len(p)),p,std)

In [ ]:
plt.plot(a,'k')
1.96*std
plt.fill_between(p-1.96*std,p+1.96*std,color='lightgreen',alpha=0.3)

In [ ]:
processed_df.get_group(('MISSOURI','PIP')).sum()

In [ ]:
detail3[1]['MISSOURI']['metrics']['ALAE']['BI'].keys()

In [ ]:
extract_detail_data(detail3,1,'MISSOURI','BI','ALAE')

#### Create a dictionary for the rounding digits

In [ ]:
rounding_dict 

In [ ]:
from collections import defaultdict
rounding_dict = defaultdict(int)
for v in variables:
    if v.endswith('Count'):
        rounding_dict[v]=0
    elif v.endswith('CNT'):
        rounding_dict[v]=0
    else:
        rounding_dict[v]=2

In [ ]:
vt_mpc = processed_df.get_group(('VERMONT','MPC'))
np.shape(vt_mpc)

In [ ]:
vt_mpc.SUIT_CNT.value_counts()

In [ ]:
build_models_for_group(processed_df.get_group(('MISSOURI','PIP')),'MISSOURI','PIP',variables)

In [ ]:
build_models_for_group(processed_df.get_group(('VERMONT','MPC')),'VERMONT','MPC',variables)

In [ ]:
def build_models_for_group(df,state,coverage,variables_of_interest,rnd_digits=rounding_dict): 
    #,,my_time_horizons,my_panel,detail_data,overview_data):
    my_time_horizons =[1,6,12]
    print state, coverage, len(df)
    for time_horizon in my_time_horizons:
        X, y_data, x, undiff = time_series_to_cross_section(df[variables_of_interest],
                                                        forecast_horizon=time_horizon,
                                                        max_fair_lags=13,
                                                        seasonal_factor=12)
        for variable in variables_of_interest:
            # Account for time series with no variation
            state_actuals = df[variable]
            std_of_actuals = state_actuals.std(ddof=1)
            if std_of_actuals == 0:
                print "\t",time_horizon,variable, " is zero"
                continue
            
            """
                # code to deal with forecast data
                value_col = 'v'+str(time_horizon)
                error_col = 'e'+str(time_horizon)
                if std_of_actuals == 0:
                    overview_data[time_horizon]["metrics"][variable][coverage][state] = [0]*len(state_actuals)
                    detail_data[time_horizon][state]["metrics"][variable][coverage] = {"stddev": 0,
                                                                     "actual": list(state_actuals),
                                                                     "predicted" : list(state_actuals)}
                    my_panel[state][coverage].loc[variable,value_col] = 0#point_forecast[0]
                    my_panel[state][coverage].loc[variable,error_col] = 0#point_forecast[1]                                                     "predicted": list(state_actuals)}
                    break
            """
            # Build custom model for dataset. This is the line of code that takes all the time.
            final_model, variables_in_model = optimized_rf(X, y_data[variable],
                                                       variable_importance_n_estimators=50,
                                                       n_estimators_in_grid_search=20,
                                                       number_of_important_variables_to_use_options=[6],#[6],#, 8, 10, 12, 15, 20],
                                                       variable_importance_max_features_options=['sqrt'],#, 0.5, .75, 'auto'],
                                                       n_estimators_to_retrain_best_model=50,
                                                       verbose=False, n_random_models_to_test=3,
                                                       charts=False, n_jobs=1)



            state_predictions = undiff(final_model.oob_prediction_, variable, True)
            state_residuals = df[variable] - state_predictions
            data_consumed_for_model = sum(state_residuals == 0)
            std_of_residuals = state_residuals[data_consumed_for_model:].std(ddof=1)
            state_std_residuals = state_residuals/std_of_residuals
            abs_average_error = abs((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - df[variable][data_consumed_for_model:])/std_of_actuals).mean()
            MSE = (((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - df[variable][data_consumed_for_model:])/std_of_actuals)**2).mean()         

            #  point forecast for this variable and time horizon
            forecast = model_forecast(x,variable, final_model, variables_in_model,undiff)
            #convert list of values to 2 floating point digits or 0 for counts
            rnd_size = rnd_digits[variable]
            if rnd_size > 0 :
                state_actuals = [ round(elem,rnd_size) for elem in list(state_actuals)]
                state_predictions = [ round(elem,rnd_size) for elem in list(state_predictions)]
            else:
                state_actuals = [ int(round(elem,0)) for elem in list(state_actuals)]
                state_predictions = [ int(round(elem,0)) for elem in list(state_predictions)]
            
            overview_values = [round(elem,2) for elem in list(state_std_residuals.values)]
            
            
            #    # Update primary data structures with model results
            #    overview_data[time_horizon]["metrics"][variable][coverage][state] = overview_values
            #    detail_data[time_horizon][state]["metrics"][variable][coverage] = {"stddev": round(std_of_residuals,5),
            #                                                     "actual": state_actuals,
            #                                                     "predicted": state_predictions}
            MSE = round(MSE,5)
            abs_average_error = round(abs_average_error,5)
            #    my_panel[state][coverage].loc[variable,value_col] = round(forecast,5)
            #    my_panel[state][coverage].loc[variable,error_col] = round(std_of_residuals,5)
            #    error_dict[time_horizon].append([MSE,abs_average_error])
            print "\t",time_horizon,variable,len(overview_values),MSE,abs_average_error
        #print '********************************************************************************************'
    #print total_absolute_average_error / float(models_built), total_mean_squared_error / float(models_built)
    #print models_built
    #print '********************************************************************************************'
    
    return #(my_panel,detail_data,overview_data,error_dict)

In [ ]:
states = list(data.STATE.unique())
coverages = list(data.COVERAGE.unique())
len(states),len(coverages),len(variables_of_interest)

In [ ]:
processed_ALABI = processed_df.get_group(('ALABAMA','BI'))
orig_ALABI = dd[(dd.STATE == 'ALABAMA') &(dd.COVERAGE == 'BI')]
sum(processed_ALABI['CWP']==orig_ALABI['CWP']) == len(orig_ALABI)

In [ ]:
r12_ALABI = r12df[(r12df.STATE == 'ALABAMA') & (r12df.COVERAGE == 'BI')]
processed_ALABI.CIF.plot()
orig_ALABI.CIF.plot()
r12_ALABI.CIF.plot()


In [ ]:
r12_ALABI.head()

# CLEANING The results
* need to remove nans from json data files

In [ ]:
data[(data.STATE == 'VERMONT')& (data.COVERAGE =='MPC')]

In [ ]:
for variable in variables:
    print variable
    print detail3[1]['VERMONT']['metrics'][variable]['MPC'].keys()

In [ ]:
extract_detail_data(detail3,1,'VERMONT','MPC','Paid Count')

In [ ]:
extract_detail_data(detail3,1,'VERMONT','COLL','Indemnity')

# Begin to look at this data

In [ ]:
Roverview3[1]['table']

In [ ]:
Rdetail3[1]['GEORGIA']['metrics'].keys()

In [ ]:
variables

In [ ]:
def detail_to_data_frame(detail,time_horizon,state,coverage,metric,r12_flag =False):
    data = detail[time_horizon][state]['metrics'][coverage][metric]
    start_month = detail[time_horizon][state]['startMonth']
    start_year = detail[time_horizon][state]['startYear']
    actual = data['actual']
    predicted = data['predicted']
    std = data['actual']
    nmonths = len(actual)
    forecast_values = list(np.zeros((12,)))
    forecast_errors = list(np.zeros((12,)))
    if time_horizon == 1:
        forecast_values = data['forecast']['values']
        forecast_errors = data['forecast']['std']
    #if r12_flag
    return actual,predicted,std,forecast_values, forecast_errors

In [ ]:
#len(
len(Rdetail3[1]['GEORGIA']['metrics']['ALAE']['Property']['actual'])

In [ ]:
type(Rdetail3[1]['GEORGIA']['startMonth']), Rdetail3[1]['GEORGIA']['startYear']

In [ ]:
pd.date_range?

In [ ]:
start_month = 0
start_year = 2007

In [ ]:
start_date = str(start_month+12)+'-'+str(start_year)
pd.date_range(start_date,freq='M',periods=87)

In [ ]:
12*8+3

### use hierarchical grouping to get the state-coverage-combo grouped dataframe

In [ ]:
gdf = data.groupby(('STATE','COVERAGE'))
len(gdf)
#my_grouped_data = data.groupby((x, y))  # set_index([x,y,t]

### in order to run the below example, pull out a specific state,coverage

In [ ]:
test_case1 = gdf.get_group(('ALABAMA','BI'))
test_case1.head()

In [ ]:
X, y_data, x, undiff = time_series_to_cross_section0(test_case1[variables_of_interest], 
                                                            forecast_horizon=1, 
                                                            max_fair_lags=2, 
                                                            seasonal_factor=12)

In [ ]:
np.shape(test_case1[variables_of_interest])

In [ ]:
np.shape(X)

In [ ]:
np.shape(y_data), np.shape(x),np.shape(undiff)

In [ ]:
X.head()

In [ ]:
X6, y_data6, x6, undiff6 = time_series_to_cross_section0(test_case1[variables_of_interest], 
                                                            forecast_horizon=6, 
                                                            max_fair_lags=6, 
                                                            seasonal_factor=12)

In [ ]:
X12, y_data12, x12, undiff12 = time_series_to_cross_section0(test_case1[variables_of_interest], 
                                                            forecast_horizon=12, 
                                                            max_fair_lags=12, 
                                                            seasonal_factor=12)

### take a peak at some of the original timeseries here


In [ ]:
ts_case1 = test_case1.copy()
ts_case1.index = ts_case1.YEAR
ts_case1.drop(['COVERAGE','STATE','YEAR'],axis=1,inplace=True)
ts_case1.head()

In [ ]:
x.values

# Looking at getting A) extended forecast horizon and B) additional predictions

### apply the rf prediction to it

In [ ]:
def make_prediction_for_ts(focused_data,variables_of_interest,X,y_data,x,undiff):
    verbose=True
    total_absolute_average_error = 0
    total_mean_squared_error = 0
    models_built = 0
    variable_models_built = 0
    for variable in variables_of_interest[:-2]:
        # Housekeeping variables
        models_built += 1
        # Account for time series with no variation
        state_actuals = focused_data[variable]
        std_of_actuals = state_actuals.std(ddof=1)
        if std_of_actuals == 0:
            overview_data["metrics"][variable][coverage][state] = [0]*len(state_actuals)
            detail_data[state]["metrics"][variable][coverage] = {  "stddev": 0,
                                                                       "actual": list(state_actuals),
                                                                       "predicted": list(state_actuals)}
            break
                
        variable_models_built += 1
            
        # Build custom model for dataset. This is the line of code that takes all the time.
        final_model, variables_in_model = optimized_rf(X, y_data[variable], 
                                                           variable_importance_n_estimators=20, 
                                                           n_estimators_in_grid_search=10,
                                                           number_of_important_variables_to_use_options=[6],#, 8, 10, 12, 15, 20],
                                                           variable_importance_max_features_options=['sqrt'],#, 0.5, .75, 'auto'],
                                                           n_estimators_to_retrain_best_model=10, 
                                                           verbose=False, n_random_models_to_test=1,
                                                           charts=False, n_jobs=1)

        state_predictions = undiff(final_model.oob_prediction_, variable, True)
            
        state_residuals = focused_data[variable] - state_predictions
        data_consumed_for_model = sum(state_residuals == 0)
        std_of_residuals = state_residuals[data_consumed_for_model:].std(ddof=1)
        state_std_residuals = state_residuals/std_of_residuals
        abs_average_error = abs((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals).mean()
        MSE = (((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals)**2).mean()
        total_absolute_average_error += abs_average_error
        total_mean_squared_error += MSE
            
        # Update primary data structures with model results
        overview_data["metrics"][variable][coverage][state] = list(state_std_residuals.values)
        detail_data[state]["metrics"][variable][coverage] = {"stddev": std_of_residuals,
                                                             "actual": list(state_actuals),
                                                             "predicted": list(state_predictions.round(0).astype(int))
                                                             }
        
        if verbose:
            figsize(15,5)
            title("%s %s: %s" % (state, coverage, variable), fontsize=16)
                # plot(state_predictions)
            plot(state_actuals, "k")
            plot(state_predictions + 1.96*std_of_residuals, "0.5")
            LCL = state_predictions - 1.96*std_of_residuals
            if sum(state_actuals < 0) == 0:
                LCL[LCL<0] = 0
            plot(LCL, "0.5")
            show()

        print "OOB R^2: %f" % final_model.oob_score_
        print "Absolute average error: %f" % abs_average_error
        print "Mean squared error: %f\n" % MSE
    """
        output = {'state': state,
                  'variable': variable,
                  'coverage': coverage,
                  'overview_data': list(state_std_residuals.values),
                  'detail_data': {"stddev": std_of_residuals,
                                  "actual": list(state_actuals),
                                  "predicted": list(state_predictions.round(0).astype(int))},
                  'abs_average_error': abs_average_error,
                  'MSE': MSE}

        print json.dumps(output)
        """    
    #print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)
    return(state_predictions,final_model,variables_in_model,variable)

In [ ]:
make_prediction_for_ts(test_case1,variables_of_interest,X,y_data,x,undiff)

In [ ]:
mdl.feature_importances_, mdl_vars

In [ ]:
mdl6.feature_importances_, mdl_vars6

In [ ]:
mdl12.feature_importances_, mdl_vars12

In [ ]:
a,mdl,mdl_vars,var = make_prediction_for_ts(test_case1,variables_of_interest, X,y_data,x,undiff)

In [ ]:
var

In [ ]:
x[mdl_vars], var

In [ ]:
raw_predict = mdl.apply(x[mdl_vars])

undiff(raw_predict, var, True)

In [ ]:
a,mdl,mdl_vars,var = make_prediction_for_ts(test_case1,variables_of_interest, X,y_data,x,undiff)

In [ ]:
print x, mdl_vars

raw_predict = mdl.apply(x[mdl_vars])

undiff(raw_predict, 'Indemnity', True)


In [ ]:
print raw_predict

In [ ]:
vals_idemnity = [7823452.88,  7823439.88,  7823469.88,  7823439.88,  7823460.88,7823457.88,  7823453.88,  7823468.88,  7823466.88,  7823471.88]


In [ ]:
ts_case1['Indemnity_predict'] = a
#ts_case1.ix[len(ts_case1),'forecast'] = np.mean(vals_idemnity)

In [ ]:
ts_case1.index[-1]

In [ ]:
import datetime
x_date1 = datetime.date(2015,2,1)
print x_date1

In [ ]:
ts_case1.Indemnity.plot(color='steelblue',label='actual')
ts_case1.Indemnity_predict.plot(color='darkred',style=':',marker='s',label='prediction')

#now add the vals_idemnity

#for my_yval in vals_idemnity:
#    plt.plot(x_date1,my_yval,'s',color='indianred',alpha=0.3)
    
#plt.show()

In [ ]:
for my_yval in vals_idemnity:
    plt.plot(x_date1,my_yval,'ok')

In [ ]:
len(a), test_case1.head()

In [ ]:
plt.scatter(test_case1.Overall, a,color='forestgreen',alpha=0.6)

In [ ]:
ts_case1['Overall_predict']=a
ts_case1.Overall.plot()
ts_case1.Overall_predict.plot(marker='o',style=':',color='darkred')

### okay how to export the forecast?

In [ ]:
a6,mdl6,mdl_vars6,var6 = make_prediction_for_ts(test_case1,variables_of_interest, X6,y_data6,x6,undiff6)

In [ ]:
var6

In [ ]:
mdl_vars6

In [ ]:
def apply_model(x,variable, mdl,mdl_vars,undiff):
    raw_predict = mdl.apply(x[mdl_vars])
    forecast = undiff(raw_predict, variable, True)
    return forecast,raw_predict

In [ ]:
fore6,raw6 = apply_model(x6,var6,mdl6,mdl_vars6,undiff6)

In [ ]:
apply_model(x,var,mdl,mdl_vars,undiff)

In [ ]:
fore6[0]

In [ ]:
start_date = datetime.date(2007,1,1)
end_date = datetime.date(2015,1,1)
years = matplotlib.dates.YearLocator()
fig,ax = plt.subplots()
ax.plot(ts_case1.Indemnity.values)
ax.xaxis.set_major_locator(years)
plt.show()

In [ ]:
a12,mdl12,mdl_vars12,var12 = make_prediction_for_ts(test_case1,variables_of_interest, X12,y_data12,x12,undiff12)

In [ ]:
fore,raw = apply_model(x,var,mdl,mdl_vars,undiff)
fore

In [ ]:
fore12,raw12 = apply_model(x12,var12,mdl12,mdl_vars12,undiff12)

In [ ]:
fore12

In [ ]:
my_dates = []
for m in xrange(1,13):
    my_dates.append(datetime.date(2014,m,1))

my_dates.append(end_date)
len(my_dates), my_dates
yv1 = ts_case1.Indemnity[-13:].values
#yv1
yv2 = ts_case1.Indemnity_predict[-13:].values

In [ ]:
x_date2 = datetime.date(2015,7,1)
x_date3 = datetime.date(2016,1,1)
for my_yval in fore[0]:
    plt.plot(x_date1,my_yval,'dk')
    
for y in fore6[0]:
    plt.plot(x_date2,y,'dg')

for y in fore12[0]:
    plt.plot(x_date3,y,'dr')

    
plt.plot(my_dates,yv1,color='steelblue',marker='*')
plt.plot(my_dates,yv2,'o:',color='deeppink')

In [ ]:
def make_prediction_for_ts2(focused_data,variables_of_interest,X,y_data,x,undiff):
    verbose=True
    total_absolute_average_error = 0
    total_mean_squared_error = 0
    models_built = 0
    variable_models_built = 0
    max_forecast_horizon = 12 # the number of months to forecast
    for variable in variables_of_interest[:-2]:
        # Housekeeping variables
        models_built += 1
        # Account for time series with no variation
        state_actuals = focused_data[variable]
        std_of_actuals = state_actuals.std(ddof=1)
        if std_of_actuals == 0:
            overview_data["metrics"][variable][coverage][state] = [0]*len(state_actuals)
            # augment with forecast
            # just use the last value of the state_actuals:
            forecast_values = state_actuals[-1]*max_forecast_horizon
            detail_data[state]["metrics"][variable][coverage] = {  "stddev": 0,
                                                                       "actual": list(state_actuals),
                                                                       "predicted": list(state_actuals),
                                                                       "forecast": {"values":forecast_values,
                                                                                    "lower_cl": forecast_values,
                                                                                    "upper_cl": forecast_values}}
            break
                
        variable_models_built += 1
            
        # Build custom model for dataset. This is the line of code that takes all the time.
        final_model, variables_in_model = optimized_rf(X, y_data[variable], 
                                                           variable_importance_n_estimators=20, 
                                                           n_estimators_in_grid_search=10,
                                                           number_of_important_variables_to_use_options=[6],#, 8, 10, 12, 15, 20],
                                                           variable_importance_max_features_options=['sqrt'],#, 0.5, .75, 'auto'],
                                                           n_estimators_to_retrain_best_model=10, 
                                                           verbose=False, n_random_models_to_test=1,
                                                           charts=False, n_jobs=1)

        state_predictions = undiff(final_model.oob_prediction_, variable, True)
            
        state_residuals = focused_data[variable] - state_predictions
        data_consumed_for_model = sum(state_residuals == 0)
        std_of_residuals = state_residuals[data_consumed_for_model:].std(ddof=1)
        state_std_residuals = state_residuals/std_of_residuals
        abs_average_error = abs((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals).mean()
        MSE = (((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals)**2).mean()
        total_absolute_average_error += abs_average_error
        total_mean_squared_error += MSE
        # call a function to make forecast
        
        # call a function to interpolate 
        # Update primary data structures with model results
        overview_data["metrics"][variable][coverage][state] = list(state_std_residuals.values)
        detail_data[state]["metrics"][variable][coverage] = {"stddev": std_of_residuals,
                                                             "actual": list(state_actuals),
                                                             "predicted": list(state_predictions.round(0).astype(int))}
        
        if verbose:
            figsize(15,5)
            title("%s %s: %s" % (state, coverage, variable), fontsize=16)
                # plot(state_predictions)
            plot(state_actuals, "k")
            plot(state_predictions + 1.96*std_of_residuals, "0.5")
            LCL = state_predictions - 1.96*std_of_residuals
            if sum(state_actuals < 0) == 0:
                LCL[LCL<0] = 0
            plot(LCL, "0.5")
            show()

        print "OOB R^2: %f" % final_model.oob_score_
        print "Absolute average error: %f" % abs_average_error
        print "Mean squared error: %f\n" % MSE
    """
        output = {'state': state,
                  'variable': variable,
                  'coverage': coverage,
                  'overview_data': list(state_std_residuals.values),
                  'detail_data': {"stddev": std_of_residuals,
                                  "actual": list(state_actuals),
                                  "predicted": list(state_predictions.round(0).astype(int))},
                  'abs_average_error': abs_average_error,
                  'MSE': MSE}

        print json.dumps(output)
        """    
    #print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)
    return(state_predictions,final_model,variables_in_model,variable)

### STOP HERE 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
from mltimeseries import time_series_to_cross_section, optimized_rf
import json

In [ ]:
coverages, len(coverages)

In [ ]:
alabama = data[data.STATE=='ALABAMA'].copy()
print alabama.shape

In [ ]:
alaBI = alabama[alabama.COVERAGE=='BI'].copy()

In [ ]:
date = pd.to_datetime(alaBI.YEAR)
#plt.plot(alaBI.YEAR,alaBI.Severity)

In [ ]:
%%time
verbose=False

models_built = 0
total_absolute_average_error = 0
total_mean_squared_error = 0

for state in states:
    coverage_group = state_data_group.get_group(state).groupby("COVERAGE")
    for coverage, focused_data in coverage_group:
        ### Test code
        if models_built >= 500:
            break
            
        variable_models_built = 0
        X, y_data, x, undiff = time_series_to_cross_section(focused_data[variables_of_interest], 
                                                            forecast_horizon=1, 
                                                            max_fair_lags=2, 
                                                            seasonal_factor=12)
        for variable in variables_of_interest:
            # Housekeeping variables
            models_built += 1
            
            # Account for time series with no variation
            state_actuals = focused_data[variable]
            std_of_actuals = state_actuals.std(ddof=1)
            if std_of_actuals == 0:
                overview_data["metrics"][variable][coverage][state] = [0]*len(state_actuals)
                detail_data[state]["metrics"][variable][coverage] = {  "stddev": 0,
                                                                       "actual": list(state_actuals),
                                                                       "predicted": list(state_actuals)}
                break
                
            variable_models_built += 1
            
            # Build custom model for dataset. This is the line of code that takes all the time.
            final_model, variables_in_model = optimized_rf(X, y_data[variable], 
                                                           variable_importance_n_estimators=20, 
                                                           n_estimators_in_grid_search=10,
                                                           number_of_important_variables_to_use_options=[6],#, 8, 10, 12, 15, 20],
                                                           variable_importance_max_features_options=['sqrt'],#, 0.5, .75, 'auto'],
                                                           n_estimators_to_retrain_best_model=10, 
                                                           verbose=False, n_random_models_to_test=1,
                                                           charts=False, n_jobs=1)

            state_predictions = undiff(final_model.oob_prediction_, variable, True)
            
            state_residuals = focused_data[variable] - state_predictions
            data_consumed_for_model = sum(state_residuals == 0)
            std_of_residuals = state_residuals[data_consumed_for_model:].std(ddof=1)
            state_std_residuals = state_residuals/std_of_residuals
            abs_average_error = abs((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals).mean()
            MSE = (((undiff(final_model.oob_prediction_, variable, True)[data_consumed_for_model:] - focused_data[variable][data_consumed_for_model:])/std_of_actuals)**2).mean()
            total_absolute_average_error += abs_average_error
            total_mean_squared_error += MSE
            
            # Update primary data structures with model results
            overview_data["metrics"][variable][coverage][state] = list(state_std_residuals.values)
            detail_data[state]["metrics"][variable][coverage] = {"stddev": std_of_residuals,
                                                                 "actual": list(state_actuals),
                                                                 "predicted": list(state_predictions.round(0).astype(int))}
                        
            if verbose:
                figsize(15,5)
                title("%s %s: %s" % (state, coverage, variable), fontsize=16)
                # plot(state_predictions)
                plot(state_actuals, "k")
                plot(state_predictions + 1.96*std_of_residuals, "0.5")
                LCL = state_predictions - 1.96*std_of_residuals
                if sum(state_actuals < 0) == 0:
                    LCL[LCL<0] = 0
                plot(LCL, "0.5")
                show()

                print "OOB R^2: %f" % final_model.oob_score_
                print "Absolute average error: %f" % abs_average_error
                print "Mean squared error: %f\n" % MSE
                
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# max_fair_lags: 6 --> 13 (21:57:45) (Almost 22 hours to run)
# variable_importance_n_estimators: 10 --> 100 
# n_estimators_in_grid_search: 10 --> 50
# number_of_important_variables_to_use_options: [8] --> [8, 10, 12, 15]
# variable_importance_max_features_options: ['sqrt'] --> ['sqrt', 0.5, 'auto']
# n_estimators_to_retrain_best_model: 10 --> 200
# n_random_models_to_test: 1 --> 6
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# max_fair_lags: 6 --> 13 (5:30:45)
# variable_importance_n_estimators: 10 --> 100 
# n_estimators_in_grid_search: 10 --> 50
# number_of_important_variables_to_use_options: [8] --> [8, 10, 12, 15]
# variable_importance_max_features_options: ['sqrt'] --> ['sqrt', 0.5, 'auto']
# n_estimators_to_retrain_best_model: 10 --> 200
# n_random_models_to_test: 1 --> 6
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# variable_importance_n_estimators: 10 --> 150 (8:51:47)
# n_estimators_in_grid_search: 10 --> 75
# number_of_important_variables_to_use_options: [8] --> [8, 10, 12, 15, 20]
# variable_importance_max_features_options: ['sqrt'] --> ['sqrt', 0.5, 'auto']
# n_estimators_to_retrain_best_model: 10 --> 200
# n_random_models_to_test: 1 --> 8
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# variable_importance_n_estimators: 10 --> 100 (2:30:10)
# n_estimators_in_grid_search: 10 --> 50
# number_of_important_variables_to_use_options: [8] --> default
# variable_importance_max_features_options: ['sqrt'] --> ['sqrt', 0.5, 'auto']
# n_estimators_to_retrain_best_model: 10 --> 100
# n_random_models_to_test: 1 --> 4
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# variable_importance_n_estimators: 10 --> 100 (5:15)
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# n_random_models_to_test: 1 --> 2; (2:30)
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
# base model (1:45)
print total_absolute_average_error/float(models_built), total_mean_squared_error/float(models_built)

In [ ]:
models_built

In [ ]:
with open('overview5.json', 'w') as outfile:
    json.dump(overview_data, outfile)

In [ ]:
with open('detail5.json', 'w') as outfile:
    json.dump(detail_data, outfile)

In [ ]:
aa = pl_retMX(dv[:],data,maine)

In [ ]:
def extract_important(X, y, max_features):
    from sklearn.ensemble import RandomForestRegressor
    import pandas as pd
    #import RandomForestRegressor from sklearn.
    #n_estimators=100, 
    #                             max_features="auto", chart=True,
    #                             random_state=42, n_jobs=1):
    n_estimators = 100
    #chart = False
    random_state = 40
    n_jobs=1
    """Returns an array of column names sorted by rf importance"""
    model = RandomForestRegressor(n_estimators, 
                                  max_features=max_features, 
                                  random_state=random_state, 
                                  n_jobs=n_jobs)
    model.fit(X, y)
    important_variables = pd.Series(model.feature_importances_, 
                                    index=X.columns)
    important_variables.sort()
    """
    if chart:
        important_variables.plot(kind="barh", figsize=(5,15))
        plt.show()
        """
    important_variables.sort(ascending=False)
    print len(important_variables.index)
    return list(important_variables.index)
